In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [948 kB]
Hit:14 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [4]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [5]:
# Filter the data and create a new DataFrame or table to retrieve all the rows 
#where the total_votes count is equal to or greater than 20 to pick reviews that
# are more likely to be helpful and to avoid having division by zero errors later on.

total_votes_20_df = vine_df.filter('total_votes > 20')
#total_votes_20_df.show()
total_votes_20_df.count()

40815

In [6]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame
# or table to retrieve all the rows where the number of helpful_votes divided by
# total_votes is equal to or greater than 50%.

import pyspark.sql.functions as F

helpful_total_df = total_votes_20_df.withColumn('help_perct', F.round((total_votes_20_df.helpful_votes/total_votes_20_df.total_votes)*100,2))
cleaned_help_perct_df = helpful_total_df.filter('help_perct >= 50')
#cleaned_help_perct_df.show()
cleaned_help_perct_df.count()

37544

In [7]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or
# table that retrieves all the rows where a review was written as part of the
# Vine program (paid), vine == 'Y'.

paid_vine_df = cleaned_help_perct_df.filter("vine =='Y'")
#paid_vine_df.show()
paid_vine_df.count()

103

In [8]:
# Repeat Step 3, but this time retrieve all the rows where the review was not
# part of the Vine program (unpaid), vine == 'N'.

unpaid_vine_df = cleaned_help_perct_df.filter("vine =='N'")
#unpaid_vine_df.show()
unpaid_vine_df.count()

37441

In [9]:
# Determine the total number of reviews

total_reviews = cleaned_help_perct_df.count()
total_reviews

37544

In [10]:
# Determine the number of 5-star reviews

total_5_star = cleaned_help_perct_df.filter('star_rating == 5').count()
total_5_star

19791

In [11]:
# Determine the number of reviews below 5-star

total_below5 = cleaned_help_perct_df.filter('star_rating < 5').count()
total_below5

17753

In [12]:
# Determine the number of 5-star paid reviews

total_5_star_paid = paid_vine_df.filter('star_rating == 5').count()
total_5_star_paid

55

In [13]:
# Determine the number of paid reviews below 5-star

total_below5_paid = paid_vine_df.filter('star_rating < 5').count()
total_below5_paid

48

In [14]:
# Determine the number of 5-star unpaid reviews

total_5_star_unpaid = unpaid_vine_df.filter('star_rating == 5').count()
total_5_star_unpaid

19736

In [15]:
# Determine the number of unpaid reviews below 5-star

total_below5_unpaid = unpaid_vine_df.filter('star_rating < 5').count()
total_below5_unpaid

17705

In [16]:
# Determine the percentage of 5-star reviews for the two types of review(paid vs unpaid).


#find total 5 star review percentage for cleaned dataframe
perct_5 = round((total_5_star / total_reviews)*100,2)
print(f'Percent of all reviews that were 5 star: {perct_5}%')

#find total 5 star paid reviews and divide by total 5 star reviews
perct_5_paid = round((total_5_star_paid / paid_vine_df.count())*100,2)
print(f'Percent of Vine reviews that were 5 star: {perct_5_paid}%')

#find total 5 star unpaid reviews and divide by total 5 star reviews
perct_5_unpaid = round((total_5_star_unpaid / unpaid_vine_df.count())*100,2)
print(f'Percent of non-Vine reviews that were 5 star: {perct_5_unpaid}%')

Percent of all reviews that were 5 star: 52.71%
Percent of Vine reviews that were 5 star: 53.4%
Percent of non-Vine reviews that were 5 star: 52.71%
